In [1]:
import pandas as pd


In [4]:
df= pd.read_csv("data.csv")
df.head()

,Job Title,Description
0,Data Analyst,job overview were seeking a data analyst to tu...
1,Data Reporting Analyst,about wspc wspc is a cooperative of outstandin...
2,Data Analyst (Power BI/Python),data analyst power bipython employment type fu...
3,Data & Reporting Analyst,our company pharmerica overview the data repor...
4,Data Quality Analyst (Remote Opportunity),vetsez is seeking a data quality analyst teste...


In [6]:
df1=pd.read_csv('UpdatedResumeDataSet.csv')


# start

In [8]:
!pip install sentence-transformers
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 70.7 MB/s eta 0:00:00


In [9]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [10]:
resume_df = pd.read_csv("UpdatedResumeDataSet.csv")
job_df = pd.read_csv("data.csv")

print(resume_df.shape)
print(job_df.shape)

resume_df.head()

(962, 2)
(521, 2)


,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [11]:
resume_df = resume_df[['Category', 'Resume']]
job_df = job_df[['Job Title', 'Description']]

resume_df.dropna(inplace=True)
job_df.dropna(inplace=True)

resume_df.reset_index(drop=True, inplace=True)
job_df.reset_index(drop=True, inplace=True)

print("Resume Data:", resume_df.shape)
print("Job Data:", job_df.shape)

Resume Data: (962, 2)
Job Data: (521, 2)


In [12]:
model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
resume_texts = resume_df['Resume'].tolist()

resume_embeddings = model.encode(
    resume_texts,
    batch_size=32,
    show_progress_bar=True
)

print("Resume Embedding Shape:", resume_embeddings.shape)

Batches:   0%|          | 0/31 [00:00<?, ?it/s]

Resume Embedding Shape: (962, 384)


In [22]:
job_texts = job_df['Description'].tolist()

job_embeddings = model.encode(
    job_texts,
    batch_size=32,
    show_progress_bar=True
)

print("Job Embedding Shape:", job_embeddings.shape)

Batches:   0%|          | 0/17 [00:00<?, ?it/s]

Job Embedding Shape: (521, 384)


In [25]:
resume_index = 0

similarities = cosine_similarity(
    [resume_embeddings[resume_index]],
    job_embeddings
)

best_match_index = np.argmax(similarities)
best_score = similarities[0][best_match_index]

print("Resume Category:", resume_df.iloc[resume_index]['Category'])
print("Best Matching Job:", job_df.iloc[best_match_index]['Job Title'])
print("Similarity Score:", round(float(best_score), 3))

Resume Category: Data Science
Best Matching Job: Data Analyst
Similarity Score: 0.617


In [26]:
def match_resume_to_jobs(resume_text, top_k=3):

    resume_vector = model.encode([resume_text])

    scores = cosine_similarity(resume_vector, job_embeddings)[0]

    top_indices = np.argsort(scores)[::-1][:top_k]

    results = []

    for idx in top_indices:
        results.append({
            "Job Title": job_df.iloc[idx]['Job Title'],
            "Score": float(scores[idx])
        })

    return results

In [18]:
job_df['Full_Text'] = job_df['Job Title'] + " " + job_df['Description']
job_texts = job_df['Full_Text'].tolist()

job_embeddings = model.encode(job_texts, batch_size=32, show_progress_bar=True)

Batches:   0%|          | 0/17 [00:00<?, ?it/s]

In [23]:
test_resume = resume_df.iloc[5]['Resume']
match_resume_to_jobs(test_resume)

[{'Job Title': 'Business & Data Analyst', 'Score': 0.5559718608856201},
 {'Job Title': 'Data Analyst I, II, III', 'Score': 0.5273094773292542},
 {'Job Title': 'Data Analyst 1 - 3477, 3498', 'Score': 0.5238929390907288}]

In [24]:
print(resume_df.iloc[5]['Category'])

Data Science


In [27]:
pickle.dump(resume_embeddings, open("resume_embeddings.pkl", "wb"))
pickle.dump(job_embeddings, open("job_embeddings.pkl", "wb"))
pickle.dump(model, open("embedding_model.pkl", "wb"))

#Train Your Own Resume Classifier

In [28]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
resume_df['Encoded_Category'] = label_encoder.fit_transform(resume_df['Category'])

print("Number of Categories:", len(label_encoder.classes_))
print(label_encoder.classes_)

Number of Categories: 25
['Advocate' 'Arts' 'Automation Testing' 'Blockchain' 'Business Analyst'
 'Civil Engineer' 'Data Science' 'Database' 'DevOps Engineer'
 'DotNet Developer' 'ETL Developer' 'Electrical Engineering' 'HR' 'Hadoop'
 'Health and fitness' 'Java Developer' 'Mechanical Engineer'
 'Network Security Engineer' 'Operations Manager' 'PMO' 'Python Developer'
 'SAP Developer' 'Sales' 'Testing' 'Web Designing']


In [29]:
# Use Already Created Resume Embeddings
X = resume_embeddings
y = resume_df['Encoded_Category']

In [30]:
# Train/Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Training size:", len(X_train))
print("Testing size:", len(X_test))

Training size: 769
Testing size: 193


In [31]:
# Train Classifier
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(max_iter=1000)

classifier.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [32]:
# Evaluate Accuracy
from sklearn.metrics import accuracy_score, classification_report

y_pred = classifier.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Accuracy: 0.9637305699481865
                           precision    recall  f1-score   support

                 Advocate       1.00      1.00      1.00         4
                     Arts       0.58      1.00      0.74         7
       Automation Testing       0.80      0.80      0.80         5
               Blockchain       1.00      1.00      1.00         8
         Business Analyst       1.00      1.00      1.00         6
           Civil Engineer       1.00      1.00      1.00         5
             Data Science       1.00      1.00      1.00         8
                 Database       1.00      1.00      1.00         7
          DevOps Engineer       1.00      0.91      0.95        11
         DotNet Developer       1.00      0.40      0.57         5
            ETL Developer       1.00      1.00      1.00         8
   Electrical Engineering       1.00      1.00      1.00         6
                       HR       1.00      1.00      1.00         9
                   Hadoop       

In [33]:
# Create Prediction Function
def predict_resume_category(resume_text):

    vector = model.encode([resume_text])

    prediction = classifier.predict(vector)[0]

    category = label_encoder.inverse_transform([prediction])[0]

    return category

In [34]:
test_resume = resume_df.iloc[10]['Resume']
predict_resume_category(test_resume)

'Data Science'

# mprove Similarity (Do This Now)

In [35]:
job_df['Full_Text'] = job_df['Job Title'] + " " + job_df['Description']

In [36]:
job_texts = job_df['Full_Text'].tolist()

job_embeddings = model.encode(
    job_texts,
    batch_size=32,
    show_progress_bar=True
)

print("New Job Embeddings Shape:", job_embeddings.shape)

Batches:   0%|          | 0/17 [00:00<?, ?it/s]

New Job Embeddings Shape: (521, 384)


In [37]:

test_resume = resume_df.iloc[5]['Resume']
match_resume_to_jobs(test_resume)

[{'Job Title': 'Business & Data Analyst', 'Score': 0.5415740013122559},
 {'Job Title': 'Data Analyst I, II, III', 'Score': 0.5343549251556396},
 {'Job Title': 'Senior Data Science Analyst', 'Score': 0.506586492061615}]

# SKILL GAP ENGINE (No LLM)

In [47]:
master_skill_list = [
   'Python', 'Data Analysis', 'Machine Learning', 'Communication', 'Project Management', 'Deep Learning', 'SQL',
        'Tableau',
        'Java', 'C++', 'JavaScript', 'HTML', 'CSS', 'React', 'Angular', 'Node.js', 'MongoDB', 'Express.js', 'Git',
        'Research', 'Statistics', 'Quantitative Analysis', 'Qualitative Analysis', 'SPSS', 'R', 'Data Visualization',
        'Matplotlib',
        'Seaborn', 'Plotly', 'Pandas', 'Numpy', 'Scikit-learn', 'TensorFlow', 'Keras', 'PyTorch', 'NLTK', 'Text Mining',
        'Natural Language Processing', 'Computer Vision', 'Image Processing', 'OCR', 'Speech Recognition',
        'Recommendation Systems',
        'Collaborative Filtering', 'Content-Based Filtering', 'Reinforcement Learning', 'Neural Networks',
        'Convolutional Neural Networks',
        'Recurrent Neural Networks', 'Generative Adversarial Networks', 'XGBoost', 'Random Forest', 'Decision Trees',
        'Support Vector Machines',
        'Linear Regression', 'Logistic Regression', 'K-Means Clustering', 'Hierarchical Clustering', 'DBSCAN',
        'Association Rule Learning',
        'Apache Hadoop', 'Apache Spark', 'MapReduce', 'Hive', 'HBase', 'Apache Kafka', 'Data Warehousing', 'ETL',
        'Big Data Analytics',
        'Cloud Computing', 'Amazon Web Services (AWS)', 'Microsoft Azure', 'Google Cloud Platform (GCP)', 'Docker',
        'Kubernetes', 'Linux',
        'Shell Scripting', 'Cybersecurity', 'Network Security', 'Penetration Testing', 'Firewalls', 'Encryption',
        'Malware Analysis',
        'Digital Forensics', 'CI/CD', 'DevOps', 'Agile Methodology', 'Scrum', 'Kanban', 'Continuous Integration',
        'Continuous Deployment',
        'Software Development', 'Web Development', 'Mobile Development', 'Backend Development', 'Frontend Development',
        'Full-Stack Development',
        'UI/UX Design', 'Responsive Design', 'Wireframing', 'Prototyping', 'User Testing', 'Adobe Creative Suite',
        'Photoshop', 'Illustrator',
        'InDesign', 'Figma', 'Sketch', 'Zeplin', 'InVision', 'Product Management', 'Market Research',
        'Customer Development', 'Lean Startup',
        'Business Development', 'Sales', 'Marketing', 'Content Marketing', 'Social Media Marketing', 'Email Marketing',
        'SEO', 'SEM', 'PPC',
        'Google Analytics', 'Facebook Ads', 'LinkedIn Ads', 'Lead Generation', 'Customer Relationship Management (CRM)',
        'Salesforce',
        'HubSpot', 'Zendesk', 'Intercom', 'Customer Support', 'Technical Support', 'Troubleshooting',
        'Ticketing Systems', 'ServiceNow',
        'ITIL', 'Quality Assurance', 'Manual Testing', 'Automated Testing', 'Selenium', 'JUnit', 'Load Testing',
        'Performance Testing',
        'Regression Testing', 'Black Box Testing', 'White Box Testing', 'API Testing', 'Mobile Testing',
        'Usability Testing', 'Accessibility Testing',
        'Cross-Browser Testing', 'Agile Testing', 'User Acceptance Testing', 'Software Documentation',
        'Technical Writing', 'Copywriting',
        'Editing', 'Proofreading', 'Content Management Systems (CMS)', 'WordPress', 'Joomla', 'Drupal', 'Magento',
        'Shopify', 'E-commerce',
        'Payment Gateways', 'Inventory Management', 'Supply Chain Management', 'Logistics', 'Procurement',
        'ERP Systems', 'SAP', 'Oracle',
        'Microsoft Dynamics', 'Tableau', 'Power BI', 'QlikView', 'Looker', 'Data Warehousing', 'ETL',
        'Data Engineering', 'Data Governance',
        'Data Quality', 'Master Data Management', 'Predictive Analytics', 'Prescriptive Analytics',
        'Descriptive Analytics', 'Business Intelligence',
        'Dashboarding', 'Reporting', 'Data Mining', 'Web Scraping', 'API Integration', 'RESTful APIs', 'GraphQL',
        'SOAP', 'Microservices',
        'Serverless Architecture', 'Lambda Functions', 'Event-Driven Architecture', 'Message Queues', 'GraphQL',
        'Socket.io', 'WebSockets'
                     'Ruby', 'Ruby on Rails', 'PHP', 'Symfony', 'Laravel', 'CakePHP', 'Zend Framework', 'ASP.NET', 'C#',
        'VB.NET', 'ASP.NET MVC', 'Entity Framework',
        'Spring', 'Hibernate', 'Struts', 'Kotlin', 'Swift', 'Objective-C', 'iOS Development', 'Android Development',
        'Flutter', 'React Native', 'Ionic',
        'Mobile UI/UX Design', 'Material Design', 'SwiftUI', 'RxJava', 'RxSwift', 'Django', 'Flask', 'FastAPI',
        'Falcon', 'Tornado', 'WebSockets',
        'GraphQL', 'RESTful Web Services', 'SOAP', 'Microservices Architecture', 'Serverless Computing', 'AWS Lambda',
        'Google Cloud Functions',
        'Azure Functions', 'Server Administration', 'System Administration', 'Network Administration',
        'Database Administration', 'MySQL', 'PostgreSQL',
        'SQLite', 'Microsoft SQL Server', 'Oracle Database', 'NoSQL', 'MongoDB', 'Cassandra', 'Redis', 'Elasticsearch',
        'Firebase', 'Google Analytics',
        'Google Tag Manager', 'Adobe Analytics', 'Marketing Automation', 'Customer Data Platforms', 'Segment',
        'Salesforce Marketing Cloud', 'HubSpot CRM',
        'Zapier', 'IFTTT', 'Workflow Automation', 'Robotic Process Automation (RPA)', 'UI Automation',
        'Natural Language Generation (NLG)',
        'Virtual Reality (VR)', 'Augmented Reality (AR)', 'Mixed Reality (MR)', 'Unity', 'Unreal Engine', '3D Modeling',
        'Animation', 'Motion Graphics',
        'Game Design', 'Game Development', 'Level Design', 'Unity3D', 'Unreal Engine 4', 'Blender', 'Maya',
        'Adobe After Effects', 'Adobe Premiere Pro',
        'Final Cut Pro', 'Video Editing', 'Audio Editing', 'Sound Design', 'Music Production', 'Digital Marketing',
        'Content Strategy', 'Conversion Rate Optimization (CRO)',
        'A/B Testing', 'Customer Experience (CX)', 'User Experience (UX)', 'User Interface (UI)', 'Persona Development',
        'User Journey Mapping', 'Information Architecture (IA)',
        'Wireframing', 'Prototyping', 'Usability Testing', 'Accessibility Compliance', 'Internationalization (I18n)',
        'Localization (L10n)', 'Voice User Interface (VUI)',
        'Chatbots', 'Natural Language Understanding (NLU)', 'Speech Synthesis', 'Emotion Detection',
        'Sentiment Analysis', 'Image Recognition', 'Object Detection',
        'Facial Recognition', 'Gesture Recognition', 'Document Recognition', 'Fraud Detection',
        'Cyber Threat Intelligence', 'Security Information and Event Management (SIEM)',
        'Vulnerability Assessment', 'Incident Response', 'Forensic Analysis', 'Security Operations Center (SOC)',
        'Identity and Access Management (IAM)', 'Single Sign-On (SSO)',
        'Multi-Factor Authentication (MFA)', 'Blockchain', 'Cryptocurrency', 'Decentralized Finance (DeFi)',
        'Smart Contracts', 'Web3', 'Non-Fungible Tokens (NFTs)',
        'Artificial Intelligence (AI)', 'Machine Learning', 'Deep Learning', 'Natural Language Processing (NLP)',
        'Computer Vision'
]

In [59]:
def extract_skills(text, skill_list):

    if not isinstance(text, str):
        return []

    text = text.lower()

    found_skills = []

    for skill in skill_list:
        if skill.lower() in text:
            found_skills.append(skill.lower())

    return list(set(found_skills))

In [61]:
job_df['Required_Skills'] = job_df['Description'].apply(
    lambda x: extract_skills(x, master_skill_list)
)

job_df[['Job Title', 'Required_Skills']].head()

,Job Title,Required_Skills
0,Data Analyst,"[logistics, power bi, reporting, r, etl, commu..."
1,Data Reporting Analyst,"[project management, power bi, hbase, sales, r..."
2,Data Analyst (Power BI/Python),"[project management, segment, power bi, r, qua..."
3,Data & Reporting Analyst,"[spring, reporting, microsoft dynamics, commun..."
4,Data Quality Analyst (Remote Opportunity),"[kubernetes, linux, communication, unity, trou..."


In [62]:
def detect_skill_gap_for_job(resume_text, job_index):

    resume_skills = extract_skills(resume_text, master_skill_list)

    required_skills = job_df.iloc[job_index]['Required_Skills']

    missing_skills = list(set(required_skills) - set(resume_skills))

    return {
        "resume_skills": resume_skills,
        "required_skills": required_skills,
        "missing_skills": missing_skills
    }

In [63]:
test_resume = resume_df.iloc[5]['Resume']

# Get similarity scores
similarities = cosine_similarity(
    model.encode([test_resume]),
    job_embeddings
)[0]

best_job_index = similarities.argmax()

print("Best Job Match:", job_df.iloc[best_job_index]['Job Title'])

detect_skill_gap_for_job(test_resume, best_job_index)

Best Job Match: Business & Data Analyst


{'resume_skills': ['machine learning', 'html', 'r', 'python'],
 'required_skills': ['power bi',
  'hive',
  'linux',
  'communication',
  'python',
  'machine learning',
  'text mining',
  'predictive analytics',
  'mapreduce',
  'sql',
  'statistics',
  'r',
  'tableau'],
 'missing_skills': ['power bi',
  'hive',
  'linux',
  'communication',
  'predictive analytics',
  'text mining',
  'mapreduce',
  'sql',
  'statistics',
  'tableau']}

In [64]:
# Save Classifier + Encoder
import pickle

pickle.dump(classifier, open("resume_classifier.pkl", "wb"))
pickle.dump(label_encoder, open("label_encoder.pkl", "wb"))

In [65]:
# Save Job Embeddings
pickle.dump(job_embeddings, open("job_embeddings.pkl", "wb"))

In [67]:
# Save Job Data With Required Skills
job_df.to_csv("processed_jobs.csv", index=False)
model.save("sentence_model")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

In [68]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence_model")

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

#  Resume Parsing

In [69]:
import re
import pandas as pd

In [70]:
def extract_email(text):

    if not isinstance(text, str):
        return None

    pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    match = re.search(pattern, text)

    return match.group() if match else None

In [71]:
def extract_phone(text):

    if not isinstance(text, str):
        return None

    pattern = r'(\+?\d[\d\s\-]{8,15}\d)'
    match = re.search(pattern, text)

    return match.group() if match else None

In [72]:
def extract_name(text):

    if not isinstance(text, str):
        return None

    lines = text.strip().split('\n')

    for line in lines[:5]:
        if len(line.split()) <= 4 and not any(char.isdigit() for char in line):
            return line.strip()

    return None

In [73]:
def extract_education(text):

    education_keywords = [
        "bachelor", "master", "phd",
        "bsc", "msc", "bs", "ms",
        "engineering", "university", "college", 'Computer Science', 'Information Technology', 'Software Engineering', 'Electrical Engineering', 'Mechanical Engineering', 'Civil Engineering',
        'Chemical Engineering', 'Biomedical Engineering', 'Aerospace Engineering', 'Nuclear Engineering', 'Industrial Engineering', 'Systems Engineering',
        'Environmental Engineering', 'Petroleum Engineering', 'Geological Engineering', 'Marine Engineering', 'Robotics Engineering', 'Biotechnology',
        'Biochemistry', 'Microbiology', 'Genetics', 'Molecular Biology', 'Bioinformatics', 'Neuroscience', 'Biophysics', 'Biostatistics', 'Pharmacology',
        'Physiology', 'Anatomy', 'Pathology', 'Immunology', 'Epidemiology', 'Public Health', 'Health Administration', 'Nursing', 'Medicine', 'Dentistry',
        'Pharmacy', 'Veterinary Medicine', 'Medical Technology', 'Radiography', 'Physical Therapy', 'Occupational Therapy', 'Speech Therapy', 'Nutrition',
        'Sports Science', 'Kinesiology', 'Exercise Physiology', 'Sports Medicine', 'Rehabilitation Science', 'Psychology', 'Counseling', 'Social Work',
        'Sociology', 'Anthropology', 'Criminal Justice', 'Political Science', 'International Relations', 'Economics', 'Finance', 'Accounting', 'Business Administration',
        'Management', 'Marketing', 'Entrepreneurship', 'Hospitality Management', 'Tourism Management', 'Supply Chain Management', 'Logistics Management',
        'Operations Management', 'Human Resource Management', 'Organizational Behavior', 'Project Management', 'Quality Management', 'Risk Management',
        'Strategic Management', 'Public Administration', 'Urban Planning', 'Architecture', 'Interior Design', 'Landscape Architecture', 'Fine Arts',
        'Visual Arts', 'Graphic Design', 'Fashion Design', 'Industrial Design', 'Product Design', 'Animation', 'Film Studies', 'Media Studies',
        'Communication Studies', 'Journalism', 'Broadcasting', 'Creative Writing', 'English Literature', 'Linguistics', 'Translation Studies',
        'Foreign Languages', 'Modern Languages', 'Classical Studies', 'History', 'Archaeology', 'Philosophy', 'Theology', 'Religious Studies',
        'Ethics', 'Education', 'Early Childhood Education', 'Elementary Education', 'Secondary Education', 'Special Education', 'Higher Education',
        'Adult Education', 'Distance Education', 'Online Education', 'Instructional Design', 'Curriculum Development'
        'Library Science', 'Information Science', 'Computer Engineering', 'Software Development', 'Cybersecurity', 'Information Security',
        'Network Engineering', 'Data Science', 'Data Analytics', 'Business Analytics', 'Operations Research', 'Decision Sciences',
        'Human-Computer Interaction', 'User Experience Design', 'User Interface Design', 'Digital Marketing', 'Content Strategy',
        'Brand Management', 'Public Relations', 'Corporate Communications', 'Media Production', 'Digital Media', 'Web Development',
        'Mobile App Development', 'Game Development', 'Virtual Reality', 'Augmented Reality', 'Blockchain Technology', 'Cryptocurrency',
        'Digital Forensics', 'Forensic Science', 'Criminalistics', 'Crime Scene Investigation', 'Emergency Management', 'Fire Science',
        'Environmental Science', 'Climate Science', 'Meteorology', 'Geography', 'Geomatics', 'Remote Sensing', 'Geoinformatics',
        'Cartography', 'GIS (Geographic Information Systems)', 'Environmental Management', 'Sustainability Studies', 'Renewable Energy',
        'Green Technology', 'Ecology', 'Conservation Biology', 'Wildlife Biology', 'Zoology'
    ]

    text_lower = text.lower()

    found = []

    for word in education_keywords:
        if word in text_lower:
            found.append(word)

    return list(set(found))

In [74]:
def extract_experience_years(text):

    pattern = r'(\d+)\+?\s*(years|year)'
    matches = re.findall(pattern, text.lower())

    if matches:
        years = [int(match[0]) for match in matches]
        return max(years)

    return None

In [75]:
def extract_skills_from_resume(text):

    if not isinstance(text, str):
        return []

    text = text.lower()

    found_skills = []

    for skill in master_skill_list:
        if skill.lower() in text:
            found_skills.append(skill.lower())

    return list(set(found_skills))

In [76]:
def parse_resume(text):

    return {
        "name": extract_name(text),
        "email": extract_email(text),
        "phone": extract_phone(text),
        "skills": extract_skills_from_resume(text),
        "education": extract_education(text),
        "experience_years": extract_experience_years(text)
    }

In [77]:
test_resume = resume_df.iloc[5]['Resume']

parsed_data = parse_resume(test_resume)

parsed_data

{'name': 'Data Science',
 'email': None,
 'phone': None,
 'skills': ['machine learning', 'html', 'r', 'python'],
 'education': ['ms', 'college'],
 'experience_years': 1}

In [78]:
import pickle

pickle.dump(master_skill_list, open("master_skill_list.pkl", "wb"))